In [ ]:
import torch
import torchvision

import sys
import os
import argparse
import warnings
import numpy as np
#!pip install pandas
import pandas as pd

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout


from sklearn.neighbors import KernelDensity
from keras.models import load_model



print(torch.__version__)

2024-12-27 15:04:15.938261: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.2.2




https://www.kaggle.com/code/eneskosar19/intrusion-detection-system-nsl-kdd


## NSL KDD Data

### Data Preprocessing

In [8]:
file_path = "../data/nsl/"
df = pd.read_csv(file_path+"KDDTrain+.txt", header=None)
df.head()


,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [9]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])


In [10]:

df.columns = columns
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [11]:
def get_unique(df, column_name = 'attack'):
    unique_vals = df[column_name].unique()
    value_counts = df[column_name].value_counts()
    print(f"Unique Values ({len(unique_vals)}): {unique_vals}\n")
    print(f"Value Counts:\n{value_counts}\n{'='*40}\n")
get_unique(df)

Unique Values (23): ['normal' 'neptune' 'warezclient' 'ipsweep' 'portsweep' 'teardrop' 'nmap'
 'satan' 'smurf' 'pod' 'back' 'guess_passwd' 'ftp_write' 'multihop'
 'rootkit' 'buffer_overflow' 'imap' 'warezmaster' 'phf' 'land'
 'loadmodule' 'spy' 'perl']

Value Counts:
attack
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: count, dtype: int64



In [12]:
attack_n = []
for i in df.attack :
  if i == 'normal':
    attack_n.append("normal")
  else:
    attack_n.append("attack")
df['attack'] = attack_n

get_unique(df)

Unique Values (2): ['normal' 'attack']

Value Counts:
attack
normal    67343
attack    58630
Name: count, dtype: int64



In [13]:
cat_features = df.select_dtypes(include='object').columns
cat_features

Index(['protocol_type', 'service', 'flag', 'attack'], dtype='object')

In [14]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
#clm=['protocol_type', 'service', 'flag', 'attack']
for x in cat_features:
    print(x)
    df[x]=le.fit_transform(df[x])

protocol_type
service
flag
attack


In [15]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,1,20,9,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,1,20
1,0,2,44,9,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,1,15
2,0,1,49,5,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,0,19
3,0,1,24,9,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,1,21
4,0,1,24,9,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,21


In [16]:
# Save the DataFrame to a CSV file
df.to_csv('../data/nsl/train.csv', index=False)

In [7]:
df = pd.read_csv('../data/nsl/train.csv')
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,1,20,9,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,11,20
1,0,2,44,9,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,11,15
2,0,1,49,5,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,9,19
3,0,1,24,9,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,11,21
4,0,1,24,9,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,1,49,5,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,9,20
125969,8,2,49,9,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,11,21
125970,0,1,54,9,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,11,18
125971,0,1,30,5,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,9,20


In [40]:
#get_unique(df, 'flag')
from sklearn.model_selection import train_test_split

y = df["attack"]
X = df.drop(["attack"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1,random_state=43) 
Y_train, Y_test = y_train, y_test 

In [43]:
np.array(X_test.iloc[0])

array([0.000e+00, 1.000e+00, 2.200e+01, 9.000e+00, 2.200e+02, 8.422e+03,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 2.100e+01, 2.200e+01,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
       9.000e-02, 2.550e+02, 2.550e+02, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 2.100e+01])

In [53]:
for x in X_test_std:
    print(x[1])
    break

-0.17725937606866238


In [ ]:
np.array(X_test_std[0])

array([-0.15213355, -0.17725938, -0.66927208,  0.66176323, -0.02138084,
        0.30014778, -0.01857777, -0.05956111, -0.01955893, -0.11135974,
       -0.14280369,  1.12254584, -0.01709063, -0.05069074, -0.01332169,
       -0.01500195, -0.01327124, -0.02458418, -0.05386776,  0.        ,
       -0.02329077, -0.17078375, -0.45127792, -0.09946711, -0.35051118,
       -0.34980039, -0.57147128, -0.56328582,  0.62896391, -0.3626009 ,
       -0.02988841,  0.65104636,  1.02113666,  0.8966801 , -0.41108508,
       -0.43122562, -0.23120588, -0.35937123, -0.35404804, -0.60248296,
       -0.56449505,  0.69554762])

In [54]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) # we use only transform in order to prevent data leakage

/Users/fuad/miniforge3/envs/nids/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


### Training

In [55]:
cat_features=['protocol_type', 'service', 'flag'] #, 'attack'
for x in cat_features:
    print(x)


X_train, Y_train, X_test, Y_test = get_data(args.dataset)

nClass = len(np.unique(Y_train))
nClass

protocol_type
service
flag


2

In [14]:
from util import get_data, get_model

class ArgsClass:
    def __init__(self):
        self.dataset = 'nsl'
        self.attack = 'fgsm'
        self.epochs = 2
        self.batch_size = 256
        #self.dataset = 'nsl'

    def my_method(self):
        print(self.dataset)

args = ArgsClass()
args.dataset


'nsl'

In [17]:

model = get_model(args.dataset, nClass)
model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
    
model.fit( X_train, Y_train, epochs=args.epochs, batch_size=args.batch_size,
    shuffle=True,
    verbose=1,
    validation_data=(X_test, Y_test)
)

Epoch 1/2


/Users/fuad/miniforge3/envs/nids/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.3505 - loss: 0.8018 - val_accuracy: 0.3611 - val_loss: 0.7921
Epoch 2/2
71/71 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3596 - loss: 0.7901 - val_accuracy: 0.3682 - val_loss: 0.7814


### crafting AE

In [56]:
model = load_model('../data/model_%s.h5' % args.dataset)
_, _, X_test, Y_test = get_data(args.dataset)
_, acc = model.evaluate(X_test, Y_test, batch_size=args.batch_size,
                        verbose=0)
print("Accuracy on the test set: %0.2f%%" % (100*acc))

Accuracy on the test set: 95.43%


In [24]:
print(type(df))
print(type(X))
print(type(X_test))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [30]:
X.iloc[9]

duration                         0.00
protocol_type                    1.00
service                         18.00
flag                             9.00
src_bytes                       26.00
dst_bytes                      157.00
land                             0.00
wrong_fragment                   0.00
urgent                           0.00
hot                              0.00
num_failed_logins                1.00
logged_in                        0.00
num_compromised                  0.00
root_shell                       0.00
su_attempted                     0.00
num_root                         0.00
num_file_creations               0.00
num_shells                       0.00
num_access_files                 0.00
num_outbound_cmds                0.00
is_host_login                    0.00
is_guest_login                   1.00
count                            1.00
srv_count                        1.00
serror_rate                      0.00
srv_serror_rate                  0.00
rerror_rate 

In [57]:
print(type(X_test))
X_test[0]

<class 'numpy.ndarray'>


array([-0.15242604, -0.17268598, -0.67105988,  0.66218613, -0.02143623,
        0.05989523, -0.0166528 , -0.05865794, -0.02175725, -0.1094489 ,
       -0.14432187,  1.12440261, -0.01747047, -0.05111607, -0.0124115 ,
       -0.01551299, -0.01294707, -0.02640452, -0.05442373,  0.        ,
       -0.02234455, -0.17179203, -0.50566862, -0.13468259, -0.34902324,
       -0.34771537, -0.57163491, -0.56383541,  0.62985275, -0.36116469,
        0.04463079,  0.65047046,  1.0229568 ,  0.89547018, -0.4084097 ,
       -0.43205957, -0.23165904, -0.35820582, -0.3534578 , -0.60177171,
       -0.56476639,  0.69780328])

In [58]:
from attacks import (fast_gradient_sign_method, basic_iterative_method,
                            saliency_map_method)
X_adv = fast_gradient_sign_method(
                    model, X_test, Y_test, eps=0.913, batch_size=128)
print(type(X_adv))

X_adv[0]

<class 'tensorflow.python.framework.ops.EagerTensor'>


<tf.Tensor: shape=(42,), dtype=float32, numpy=
array([-1.065426  , -1.085686  ,  0.24194008, -0.25081384,  0.8915638 ,
        0.9728952 , -0.9296528 ,  0.85434204,  0.89124274,  0.8035511 ,
        0.7686781 ,  0.21140265,  0.8955295 , -0.96411604, -0.92541146,
        0.897487  ,  0.9000529 ,  0.8865955 ,  0.85857624,  0.913     ,
        0.89065546, -1.084792  ,  0.40733135, -1.0476825 ,  0.56397676,
       -1.2607154 ,  0.3413651 ,  0.3491646 ,  1.5428528 ,  0.5518353 ,
       -0.8683692 ,  1.5634704 ,  0.10995686,  1.8084702 ,  0.5045903 ,
        0.4809404 ,  0.68134093, -1.2712058 ,  0.5595422 ,  0.31122828,
        0.34823358, -0.21519673], dtype=float32)>

In [67]:
print(X_adv.shape)
X_adv = np.array(X_adv)
print(X_adv.shape)
X_adv[0][0]

(4509, 42)
(4509, 42)


-1.065426

In [68]:
for i in range(len(X_test)):
    X_adv[i][1] = X_test[i][1]
    X_adv[i][2] = X_test[i][2]
    X_adv[i][3] = X_test[i][3]
    print(X_adv[i])
    break

[-1.065426   -0.17268598 -0.6710599   0.66218615  0.8915638   0.9728952
 -0.9296528   0.85434204  0.89124274  0.8035511   0.7686781   0.21140265
  0.8955295  -0.96411604 -0.92541146  0.897487    0.9000529   0.8865955
  0.85857624  0.913       0.89065546 -1.084792    0.40733135 -1.0476825
  0.56397676 -1.2607154   0.3413651   0.3491646   1.5428528   0.5518353
 -0.8683692   1.5634704   0.10995686  1.8084702   0.5045903   0.4809404
  0.68134093 -1.2712058   0.5595422   0.31122828  0.34823358 -0.21519673]


In [21]:
#craft_adv_samples.py

from craft_adv_samples import craft_one_type
craft_one_type(model, X_test, Y_test, args.dataset, args.attack,
                       args.batch_size)

Crafting fgsm adversarial samples...
................................... for NSL Dataset
Model accuracy on the adversarial test set: 60.68%


In [71]:

from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

In [72]:
X_adv = projected_gradient_descent(model, X_test, 0.8, 0.01, 40, np.inf)

In [73]:
X_adv

<tf.Tensor: shape=(4509, 42), dtype=float32, numpy=
array([[-0.5524258 , -0.5726857 , -0.2710603 , ..., -0.34177196,
        -0.16476679,  0.29780364],
       [-0.5524258 ,  0.22731388,  1.2304846 , ...,  0.23674965,
        -0.5948988 ,  0.3965914 ],
       [-0.5524258 ,  0.22731388,  1.3330462 , ...,  1.5784807 ,
         1.528609  ,  1.0978029 ],
       ...,
       [-0.5524258 , -0.5726857 , -0.2710603 , ..., -0.32177198,
        -0.16476679,  0.29780364],
       [ 0.06757396,  0.22731388,  1.6304842 , ..., -0.40831366,
         1.0548677 , -0.7720953 ],
       [-0.05242603, -0.3726859 , -0.2710603 , ..., -0.21016699,
        -0.18476677,  0.29780364]], dtype=float32)>

In [80]:
projected_gradient_descent(model_fn=model, x=X_test, eps=0.8, eps_iter=0.01, nb_iter=40, norm=np.inf)


<tf.Tensor: shape=(4509, 42), dtype=float32, numpy=
array([[-0.5524258 , -0.5726857 , -0.2710603 , ..., -0.34177196,
        -0.16476679,  0.29780364],
       [-0.5524258 ,  0.22731388,  1.2304846 , ...,  0.23674965,
        -0.5948988 ,  0.3965914 ],
       [-0.5524258 ,  0.22731388,  1.3330462 , ...,  1.5784807 ,
         1.528609  ,  1.0978029 ],
       ...,
       [-0.5524258 , -0.5726857 , -0.2710603 , ..., -0.32177198,
        -0.16476679,  0.29780364],
       [ 0.06757396,  0.22731388,  1.6304842 , ..., -0.40831366,
         1.0548677 , -0.7720953 ],
       [-0.05242603, -0.3726859 , -0.2710603 , ..., -0.21016699,
        -0.18476677,  0.29780364]], dtype=float32)>